In [90]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display
import IPython
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

from datetime import datetime
import talib
import pandas_ta as ta
# from talib.abstract import *
from math import *
from collections import OrderedDict
# import vectorbt as vbt    
import json
%matplotlib inline

matplotlib.rcParams.update({'font.size': 7, 'lines.linewidth': 0.7, 'figure.dpi': 300})

In [55]:
df = pd.read_json(r'./freq-user-data/data/binance/BTC_USDT-1m.json').dropna()
df.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']

df['date'] = pd.to_datetime(df['timestamp'], unit='ms', utc=False)
# df.index = df['time']
# df.set_index('time', drop=True, inplace=True)
df.set_index(pd.DatetimeIndex(df["date"]), inplace=True, drop=True)
# df = df[['time', 'symbol', 'source', 'resolution', 'open', 'high', 'low', 'close', 'volume']]
# df.to_csv (r'./data/binance/BTC_USDT-5m.csv', index = None)
# df.set_index('time')
df = df.sort_index()

In [95]:
# pd.Timestamp('now').floor('D') + pd.Timedelta(-7, unit='D')

# start_time = datetime(2021,5,1)

start_time = pd.Timestamp('now') + pd.Timedelta(-4, unit='W')
# start_time
ddf = df.loc[df.index >= start_time].copy()
# ddf = df.loc[df['time'] >= start_time]
# ddf

ddf = ddf.resample('5Min').agg(
    OrderedDict([
        ('open', 'first'),
        ('high', 'max'),
        ('low', 'min'),
        ('close', 'last'),
        ('volume', 'sum'),
    ])
).dropna()

# ema = ddf.close.ewm(span=50,min_periods=0,adjust=False,ignore_na=False).mean()
# detrended = ddf.close - ddf.ema

ddf['lret'] = ddf.close.apply(np.log).diff(1)
ddf['cumlret'] = ddf.lret.cumsum()

dlen = len(ddf.index)
dlen

7972

In [167]:

sl_n2= IntSlider(description="n2", min=0, max=floor(log(dlen, 2.0)), step=1, value=9)
sl_w = IntSlider(description="w", min=0, max=floor(dlen/pow(2,9)), step=1, value=5)
def update_sl_w_range(*args):
    wsize = pow(2,sl_n2.value)
    sl_w.max = floor(dlen/wsize)*2 -1
sl_n2.observe(update_sl_w_range, 'value')

wdg_bb_ma = widgets.IntText(value=50,description='bbma:',disabled=False)
wdg_bb_stn = widgets.IntText(value=50,description='bbstn:',disabled=False)
wdg_bb_std = widgets.FloatText(value=3,step=0.2,description='bbstd:',disabled=False)

wdg_krn = widgets.FloatText(value=1,step=0.5,description='krn:',disabled=False)
wdg_krpow = widgets.FloatText(value=1, step=0.05,description='krpow:',disabled=False)
wdg_krstart = widgets.FloatText(value=1, step=1/6.,description='krstrt:',disabled=False)
wdg_krend = widgets.FloatText(value=2, step=1/6.,description='krend:',disabled=False)
wdg_krflip = widgets.Checkbox(value=False,description='krflip:',disabled=False)
wdg_krflipy = widgets.Checkbox(value=False,description='krflipy:',disabled=False)

# slCoolOff = IntSlider(description="cool", min=0, step=5, max=600, value=30)
# slDowntrendWait = IntSlider(description="dnt", min=0, max=100, value=30)

# slStopLoss = FloatSlider(description="sl", min=0, step=0.05, max=15,value=1.92)
# slTrailingStopLoss = FloatSlider(description="ts", min=0, step=0.2, max=15,value=10)
# slTakeProfit = FloatSlider(description="tp", min=0, step=0.2, max=15, value=10)


# slXrr = FloatSlider(description="xrr", min=1, step=0.2, max=20, value=2)

ui = widgets.VBox([
    widgets.HBox([sl_n2, sl_w]),
    widgets.HBox([wdg_bb_ma, wdg_bb_stn, wdg_bb_std]),
    widgets.HBox([wdg_krn, wdg_krstart, wdg_krend, wdg_krpow, wdg_krflip, wdg_krflipy])
    # widgets.HBox([slTrailingStopLoss, slStopLoss, slTakeProfit, slXrr]),
#     widgets.HBox([slStK, slStD, slStSm, slStT])
])


wst = wed = wsz = wdf = dbbf =False

def printer(w2log, w, bbma, bbstn, bbstd, krn, krstart, krend, krpow, krflip, krflipy):
    with open(f'./strat-params.json', "w") as f: f.write(json.dumps({"w2log": sl_n2.value, "w":sl_w.value, 
            # "sl":slStopLoss.value, "ts": slTrailingStopLoss.value, "tp": slTakeProfit.value 
            'bbma': wdg_bb_ma.value, 'bbstn': wdg_bb_stn.value, 'bbstd': wdg_bb_std.value,
            'krn': wdg_krn.value, 'krstart': wdg_krstart.value, 'krend': wdg_krend.value, 'krpow': wdg_krpow.value, 
            'krflip': wdg_krflip.value,'krflipy': wdg_krflipy.value,
            }))
    krnn = floor(60*krn/5)
    krx = np.linspace(krstart, krend, krnn)
    krxpi = np.power(krx , krpow)
    krxpi = np.exp(krxpi)
    kryo = np.cos(krxpi)
    kryo = kryo - np.mean(kryo)
    kry = kryo
    if krflip: kry = np.flipud(kry)
    if krflipy: kry = -(kry)
    kry = kry * 100
    # kry = kry - kry.mean

    ddf['bb_middle'] = ddf.close.ewm(span=bbma,min_periods=0,adjust=False,ignore_na=False).mean()
    ddf['bb_std'] = ddf.close.rolling(bbstn).std()
    ddf['bb_std_bw'] = ddf.bb_std * bbstd
    ddf['bb_lower'] = ddf['bb_middle'] - ddf.bb_std_bw

    global wst, wed, wsz, stoch, wdf
    wsz = floor(pow(2,w2log))
    wst = floor(w * wsz / 2)
    wed = floor(wst + wsz)
    
    # s = ( 
    #     (
    #         (trn < 1) |
    #         (                
    #             ((ddf['ma1'] - ddf['ma2c']).rolling(window=trn).min() > 0) 
    #         )
    #     ) &
    #     (ddf['open'] < ddf['ma2c']) &
    #     (ddf.xrr > xrr)
    # )
    # tentries = ddf['entries'] = s.vbt.signals.first()

    
    # wdf = ddf[max(ma1n,ma2n):].iloc[wst:wed,:]
    wdf = ddf.iloc[wst:wed,:].copy()

    wdf['krnl_conv'] = wdf.cumlret.rolling(krnn).apply(lambda v: (v * kry).sum())
    # wbbf = dbbf.loc[wdf.index]
    
    # texits = vbt.OHLCSTEX.run(
    #     tentries, 
    #     ddf['open'], 
    #     ddf['high'], 
    #     ddf['low'], 
    #     ddf['close'], 
    #     sl_stop=sl/100,
    #     tp_stop=tp/100,
    #     ts_stop=ts/100
    # #     stop_type=None, hit_price=None
    # ).exits

    # global pf;
    # pf = vbt.Portfolio.from_signals(ddf['close'], entries=tentries, exits=texits, direction='longonly', 
    #                                 init_cash=capital, fees=0.001, slippage=0.0025, freq='1m')

    # global trades
    # trades = pf.trades.records
    # trades['cap'] = trades['pnl'].cumsum() + capital
    # ddf_cap = pd.merge(ddf[["date"]], trades[["exit_idx", "cap"]], left_on=idx_ddf, right_index=False, 
    #          right_on="exit_idx", how="left", sort=False).fillna(method='ffill').set_index('date')

    
#     with open(f'./strategy-results/{datetime.now():strat_%Y-%m-%d}.log', "a") as f: 
#         f.write(json.dumps(dict({   
#             "dt": f'{datetime.now():%Y-%m-%d %HH:%MM:%SS}',
#             "period_start": f'{ddf.index[0]}',
#             "period_end": f'{ddf.index[-1]}',
#             "params": dict({ 'sl': sl, 'ts': ts, 'tp': tp, 'ma1n': ma1n, 'ma2n': ma2n, 'ma1c': ma1c, 'ma2c': ma2c, 'trn': trn, 'mrn': mrn, 'xrr': xrr
# #                         'std': std, 'stk': stk, 'stsm': stsm, 'stt': slStT.value
#                            }),
#             "strategy": "2EMASTH",
#             "profit": floor(100* (trades.iloc[-1].cap / capital - 1)) if len(trades.index) else 0. 
#         })
#     )); f.write("\n");
    
    
    # trades_profits = trades[trades['pnl'] > 0]
    # trades_losses = trades[trades['pnl'] < 0]    
    
    # ddf_entries =  ddf.iloc[trades['entry_idx'].values]
    # ddf_profits = ddf.iloc[trades_profits['exit_idx'].values]
    # ddf_losses = ddf.iloc[trades_losses['exit_idx'].values]
    
    plt.close("all")
#     fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(16,4), sharex='col', gridspec_kw={'width_ratios': [3,1]})
    fig = plt.figure(constrained_layout=True,figsize=(16,5))
    gs0 = fig.add_gridspec(1, 2, width_ratios=[3,1])

    gs00 = gs0[0,0].subgridspec(3, 1, height_ratios=[3,1,1])
    gs01 = gs0[0,1].subgridspec(4, 1)

    ax001 = fig.add_subplot(gs00[0,0])
    ax002 = fig.add_subplot(gs00[1,0], sharex=ax001)
    ax003 = fig.add_subplot(gs00[2,0], sharex=ax001)

    ax101 = fig.add_subplot(gs01[0,0])
    ax102 = fig.add_subplot(gs01[1,0])
    ax103 = fig.add_subplot(gs01[2,0])
    ax104 = fig.add_subplot(gs01[3,0])


#     wma1 = wdf['ma1']; wma2 = ma2[wdf.index]; wma3 = ma3[wdf.index];
    
    ax001.plot(wdf['close'])
    ax001.plot(wdf['low'], lw=0.5)
    ax001.plot(wdf['high'], lw=0.5)
    ax001.plot(wdf.bb_middle)
    # ax001.plot(wdf.ema*0.98, linestyle='dotted', lw=1)
    ax001.plot(wdf.bb_lower)
    # ax1.plot(wdf['rmin'], lw=0.5)
    
    ax001.fill_between(wdf.index, wdf.bb_middle*0.99, wdf.bb_middle*1.01, alpha=0.15)
    # ax1.fill_between(wdf.index, wdf['ma2'], wdf['ma2c'][wdf.index], alpha=0.15)
    ax002.plot(wdf.krnl_conv)    
    ax002.axhline(color='b', linestyle='--')
    # ax002.axhline(y=-1, linestyle='--')
    # ax002.axhline(y=-2, linestyle='--')
    # ax002.axhline(y=-4, linestyle='--')
    # ax002.axhline(y=1, linestyle='--')
    # ax002.axhline(y=2, linestyle='--')
    # ax002.axhline(y=4, linestyle='--')
    ax003.plot(wdf.bb_std)
    ax003.axhline(color='b', linestyle='--')


    ax102.plot(krx, kry)
    # ax102.plot(krx, kryo)
    ax102.axhline(color='b', linestyle='--')

    # wddf_entries = ddf_entries[ddf_entries.index.isin(wdf.index)]
    # wddf_profits = ddf_profits[ddf_profits.index.isin(wdf.index)]
    # wddf_losses = ddf_losses[ddf_losses.index.isin(wdf.index)]
    
    # wddf_trades = ddf_cap[wst:wed]
    
    
    # ax1.vlines(wddf_entries.index,wddf_entries['open'] * 0.97,wddf_entries['open'] * 1.02, lw=0.5, color='blue')
    # ax1.vlines(wddf_profits.index,wddf_profits['open'] *0.97,wddf_profits['open'] * 1.02, lw=0.5, color='green')
    # ax1.vlines(wddf_losses.index,wddf_losses['open'] * 0.97,wddf_losses['open'] * 1.02, lw=0.5, color='red')
    
    
    # ax2.plot(wdf.xrr,lw=0.5,color='green')

#     ax2.plot(wslowk,lw=0.5,color='orange')
#     ax2.plot(wslowd,lw=0.5,color='teal')
#     ax2.axhline(y=20, color='b', linestyle='--',lw=0.5)
#     ax2.axhline(y=80, color='b', linestyle='--',lw=0.5)
#     ax2.vlines(wddf_entries.index,0,100, lw=0.5, color='blue')
#     ax2.vlines(wddf_profits.index,0,100, lw=0.5, color='green')
#     ax2.vlines(wddf_losses.index,0,100, lw=0.5, color='red')

    
    # ax3.plot(ddf_cap['cap'])
    # ax4.plot(wddf_trades['cap'])
    
    plt.show()
    # print(pf.stats())
    # print(pf.returns_stats())
    

with open('./strat-params.json') as f: 
    js = json.loads(f.read())
    sl_n2.value=js["w2log"]; sl_w.value=js["w"];
    wdg_krn.value=js["krn"]; wdg_krstart.value=js['krstart'];wdg_krend.value=js["krend"]; wdg_krpow.value=js['krpow'];
    wdg_bb_ma.value = js['bbma']; wdg_bb_stn.value = js['bbstn']; wdg_bb_std.value = js['bbstd'];
    wdg_krflip.value = js['krflip']; wdg_krflipy.value = js['krflipy']
#     slStopLoss.value=js["sl"]; slTrailingStopLoss.value=js["ts"]; slTakeProfit.value=js["tp"];
    
out = widgets.interactive_output(printer, {"w2log":sl_n2, "w":sl_w, 
    'bbma': wdg_bb_ma, 'bbstn': wdg_bb_stn, 'bbstd': wdg_bb_std,
    'krn': wdg_krn, 'krstart': wdg_krstart, 'krend': wdg_krend, 'krpow': wdg_krpow, 'krflip': wdg_krflip, 'krflipy': wdg_krflipy
                                # "sl":slStopLoss, "ts": slTrailingStopLoss, "tp": slTakeProfit, 
                                        #    "ma1n": slMa1N, "ma2n": slMa2N, "ma1c": slMa1c, "ma2c": slMa2c,"trn": slTrN, 'mrn': slMrN, 'xrr': slXrr,
#                                            "stk": slStK, "std": slStD, "stsm": slStSm, 'stt': slStT
                                          })
    
# interactive_plot = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})````
# interactive(printer,w2log=sl_n2, w=sl_w, cool=slCoolOff, dnt=slDowntrendWait,
#                                 sl=slStopLoss, tgt=slTarget);
display(ui, out)

# output = interactive_plot.children[-1]
# # output.layout.width = '1200px'
# output.layout.height = '500px'
# interactive_plot


Output()

In [995]:
with open(f'./strat-params.json', "w") as f: f.write(json.dumps({"w2log": sl_n2.value, "w":sl_w.value, "sl":slStopLoss.value, "ts": slTrailingStopLoss.value, 
           "tp": slTakeProfit.value, "ma1n": slMa1N.value, "ma2n": slMa2N.value, "ma1c": slMa1c.value, "ma2c": slMa2c.value,"trn": slTrN.value, 'mrn': slMrN.value,
             'xrr': slXrr.value})) ;

#     "stk": slStK.value, "std": slStD.value, "stsm": slStSm.value, 'stt': slStT.value,
                    
# with open('./strat-params.json') as f: 
#     js = json.loads(f.read())
#     sl_n2.value=js["w2log"]; sl_w.value=js["w"]; slStopLoss.value=js["sl"]; slTrailingStopLoss.value=js["ts"]; slTrN.value=js["trn"];
#     slTakeProfit.value=js["tp"]; slMa1N.value=js["ma1n"]; slMa2N.value=js["ma2n"]; slMa1c.value=js["ma1c"]; slMa2c.value=js["ma2c"]; 
#     slStK.value=js['stk']; slStD.value=js['std']; slStSm.value=js['stsm'];



In [155]:

wdg_krn = widgets.IntText(value=50,description='krn:',disabled=False)
wdg_krpow = widgets.FloatText(value=1, step=0.05,description='krpow:',disabled=False)
wdg_krstart = widgets.FloatText(value=0, step=1/6.,description='krstrt:',disabled=False)
wdg_krend = widgets.FloatText(value=2, step=1/6.,description='krend:',disabled=False)
wdg_krflip = widgets.Checkbox(value=False,description='krflip:',disabled=False)
wdg_krflipy = widgets.Checkbox(value=False,description='krflipy:',disabled=False)
ui = widgets.VBox([
        widgets.HBox([wdg_krn, wdg_krstart, wdg_krend, wdg_krpow]),
        widgets.HBox([wdg_krflip, wdg_krflipy])
    ])
kry = False
def inter_plot(krn, krstart, krend, krpow, krflip, krflipy):
    global kry
    plt.close("all")
    krx = np.linspace(krstart, krend, krn)
    # krxpi = (krx ** krpow) * pi
    krxpi = np.power(krx , krpow)
    krxpi = np.exp(krxpi)
    kry = np.cos(krxpi)
    if krflip: kry = np.flipud(kry)
    if krflipy: kry = -(kry)
    kry = kry - np.mean(kry)
    # krnl = np.flipud(krnl) 
    fig, (ax1) = plt.subplots(1, 1,figsize=(2,0.5), sharex='col', gridspec_kw={'width_ratios': [1]})
    ax1.plot(krx, kry)
    ax1.plot(krx, np.flipud(kry))

out = widgets.interactive_output(inter_plot, {'krn': wdg_krn, 'krstart': wdg_krstart, 'krend': wdg_krend, 'krpow': wdg_krpow,
    'krflip': wdg_krflip, 'krflipy': wdg_krflipy})
display(ui, out)


Output()

In [157]:
floor(pi)

3

In [159]:
3.

3.0